# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import os
import shutil
import joblib
import logging
import csv
from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.model import Model
import requests
import json

from azureml.core import ScriptRunConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

### Workspace

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="hd-udacity-capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

### Configure Compute cluster

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = "hd-project"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=5, 
                                                           idle_seconds_before_scaledown=2400,
                                                           vm_priority='lowpriority')
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
from azureml.core import Dataset

ds = Dataset.get_by_name(ws, name='Glass Dataset')
df = ds.to_pandas_dataframe()
df

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# Specify parameter sampler
ps = RandomParameterSampling(parameter_space={"--C":uniform(0.01,1),"--max_iter": choice(50,100,150,200)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 5, slack_factor = 0.1, delay_evaluation = 1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',
              compute_target=compute_target,
              entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs= 100,
                             max_concurrent_runs=4)

In [ ]:
# Submitting the hyperdrive run to the experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.get_status()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

In [ ]:
parameter_values = best_run.get_details()['runDefinition']['arguments']
print(parameter_values)

In [ ]:
best_run_metrics = best_run.get_metrics()
print("Best_run_id",best_run.id)
print('\n Metrics: ', best_run_metrics)
print("Best_run_accuracy",best_run_metrics['Accuracy'])

In [ ]:
best_run.get_file_names()

In [ ]:
#TODO: Save the best model

best_model_hd=best_run.register_model(model_name='HyperdriveModel',model_path='outputs/hdmodel.joblib',description='best hyperdrive model')
best_model_hd

In [ ]:
print(best_model_hd)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service